In [163]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

## Data Preprocessing

In [66]:
# Load Retail Dataset
retail = pd.read_csv("Retail_2010_2011.csv")

In [67]:
# Check Dataset for Null Values
retail.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [68]:
# Remove rows where Customer ID is Null
retail= retail.dropna(subset=['Customer ID']).reset_index(drop=True)

In [69]:
# Check Dataset for Null Values
retail.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [70]:
# exclude cancelled orders
retail = retail[~retail["Invoice"].str.contains("C", na=False)]

In [71]:
#  exclude invoices where nothing was ordered 
retail = retail[retail["Quantity"] > 0]

In [76]:
# Outlier Work
def outlier_thresholds(dataframe, variable):
    Q1 = dataframe[variable].quantile(0.25)
    Q3 = dataframe[variable].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [77]:
# Replace outliers wioth extreme bounds
replace_with_thresholds(retail,"Quantity")
replace_with_thresholds(retail,"Price")

In [78]:
# Create Total Price Column
retail["TotalPrice"] = retail["Quantity"] * retail["Price"]

In [79]:
# Convert InvoiceDate into date time
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'], format="%m/%d/%Y %H:%M")

## Create Recency, Frequency, Monetary and RFM Values

In [80]:
def rfm_metrics(data):
    today_date = data["InvoiceDate"].max()
    date_with_offset = today_date + timedelta(days=2)
    rfm = data.groupby('Customer ID').agg({'InvoiceDate': lambda date: (date_with_offset - date.max()).days,
                                                'Invoice': lambda num: num.nunique(),
                                                "TotalPrice": lambda price: price.sum()})

    rfm.columns = ['recency', 'frequency', "monetary"]

    rfm = rfm[(rfm['monetary'] > 0)]
    return rfm


In [81]:
rfm= rfm_metrics(retail)

In [159]:
# RFM score
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])



# segment naming
rfm['rfm_segment'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)

seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
}

rfm['rfm_segment'] = rfm['rfm_segment'].replace(seg_map, regex=True)
rfm = rfm[["recency", "frequency", "monetary", "rfm_segment"]]

## Calculating CLTV Score

In [164]:
def create_cltv_calculated(dataframe):
    # avg_order_value
    dataframe['avg_order_value'] = dataframe['monetary'] / dataframe['frequency']

    # purchase_frequency
    dataframe["purchase_frequency"] = dataframe['frequency'] / dataframe.shape[0]

    # repeat rate & churn rate
    repeat_rate = dataframe[dataframe.frequency > 1].shape[0] / dataframe.shape[0]
    churn_rate = 1 - repeat_rate

    # profit_margin
    dataframe['profit_margin'] = dataframe['monetary'] * 0.05
    
    # Customer Value
    dataframe['cv'] = (dataframe['avg_order_value'] * dataframe["purchase_frequency"])

    # Customer Lifetime Value
    dataframe['cltv'] = (dataframe['cv'] / churn_rate) * dataframe['profit_margin']

    # minmaxscaler
    scaler = MinMaxScaler(feature_range=(1, 100))
    scaler.fit(dataframe[["cltv"]])
    dataframe["cltv_calculated"] = scaler.transform(dataframe[["cltv"]])

    dataframe["cltv_calculated_segment"] = pd.qcut(dataframe["cltv_calculated"], 3, labels=["C", "B", "A"])

    dataframe = dataframe[["recency", "frequency", "monetary", "rfm_segment",
                           "cltv_calculated", "cltv_calculated_segment"]]
    
    return dataframe

In [166]:
rfm_cltv = create_cltv_calculated(rfm)

In [167]:
rfm_cltv

,recency,frequency,monetary,rfm_segment,cltv_calculated,cltv_calculated_segment
Customer ID,,,,,,
12346.0,327,1,28.08,hibernating,1.000006,C
12347.0,3,7,3877.44,champions,1.115128,A
12348.0,76,4,557.57,at_risk,1.002381,B
12349.0,20,1,1336.65,promising,1.013681,A
12350.0,311,1,301.90,hibernating,1.000698,C
...,...,...,...,...,...,...
18280.0,279,1,174.50,hibernating,1.000233,C
18281.0,182,1,61.92,hibernating,1.000029,C
18282.0,9,2,150.96,potential_loyalists,1.000174,C


In [169]:
# export to csv
rfm.to_csv('rfm_cltv_2010_2011.csv', index=True)